In [79]:
import requests, time, re, random,hashlib
from bs4 import BeautifulSoup
import tweepy
from nltk import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Import the utilities here

In [80]:
import utilities

### We will create a class called Company to hold all the information we have about each company

In [81]:
class Company(object):

    def __init__(self, name):
        self.name = name
        self.articles = None
        self.tweets = None
        self.articles_sentiments = None
        self.tweets_sentiments = None
        self.investment = None
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)


### Now let's get some tweets

In [82]:
def get_tweets(company_name, n=10):
    
    '''Takes a company name, returns a list of words in the tweets DO NOT MODIFY THIS FUNCTION '''
    
    api = create_twitter_api()
    tweets = api.search(q=company_name,lang='en', show_user=True, count=n)
    tweet_words = get_tweets_bag_of_words(tweets)
    return tweet_words

In [83]:
def create_twitter_api():
    
    '''
        Creates a twitter api client wrapper
        create_twitter_api takes no parameters but creates and returns a twitter API wrapper
        that you can use to interact with Twitter. You will copy and paste your credentials obtained in
        the first step here. 
        
        TODO: ERASE THEM WHEN YOU TURN IN THE ASSIGNMENT.
    '''
    
    # Variables that contains the user credentials to access Twitter API 
    ACCESS_TOKEN = '18374056-dppeSHfEbmM9Fs9eDhqEoTN9UVEX8DBkdkZ2rs0ji'
    ACCESS_SECRET = 'vM9xxXKdRxwL4wRqYSEZo4I13kSKqC79sorzbeVDdW35b'
    CONSUMER_KEY = 'x83VcMHlmDUpN3QFfuc5I4WEp'
    CONSUMER_SECRET = 'dG5RE8tae3h0tbF2Lg4z4aQOaPH8b71zLVanKPqXmTKEjxaAmh'
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)
    
    return api

In [84]:
def get_tweets_bag_of_words(tweets):
    
    '''
        Takes the result returned from twitter search, returns a list of words in the tweets
        
        get_tweet_bag_of_words takes the tweet search results and returns a list of words. You
        will need to parse the tweet search results in order to get the twitter status texts, get rid of the
        hashtag and other punctuations, and find a list of words. See what’s being returned:
        https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    '''
    
    words = []
    
    for tweet in tweets:
        print(tweet._json)
        words += tweet._json['text'].split(" ")
        print
        print
    return words

### Now let's fetch some articles about this company

In [85]:
def get_article(company_name, n=3):
    
    '''
    DO NOT MODIFY THIS FUNCTION
    
    Take a company name, returns a list of words in articles associated with the company.'''
    
    results = get_search_results(company_name)
    articles = get_articles_from_search_results(results,n)
    articles_bag_of_words = get_articles_bag_of_words(articles)
    
    return articles_bag_of_words

In [86]:
def get_search_results(company_name):
    
    '''
    Return an html with search results for a given company.
    get_search_results takes the name of the company and returns the search result page of USA
    News Today of the company. Below are some search result links for different terms.
    
    In technical terms: observe the pattern, recreate your own search link, and use fetch
    function we imported to get the html content.
    https://www.usatoday.com/search/facebook/
    
    '''
    url = "https://www.usatoday.com/search/" + company_name + "/"
    result = utilities.fetch(url)
#     print(result)

    return result

In [87]:
def get_articles_from_search_results(results_html, n):
    
    '''
        Return a list of n article htmls for given search results html.
    
        get_articles_from_search_results takes the search result page, get all the links to the news
        articles, and opens n links (default to 3 articles: for testing we suggest you use a small number
        because you will get your results faster and you are not aggressively scraping content and
        risking getting banned). Not all links returned from the page will be links to articles, so your
        function should ensure the links selected are in fact articles from the company search. In
        technical terms: find all the links to articles on the page (<a> using BeautifulSoup), and
        use fetch function we imported to get the html content of each link, store the article
        htmls, return a list of html.
    '''
    
    domain = 'http://www.usatoday.com'
    articles = []
    
    soup = BeautifulSoup(results_html)
    links = []
    for link in soup.find_all('a', href=True):
        if link['href'][0:7] == '/story/':
            links.append(link['href'])
    
    for idx, link in enumerate(links):
        if idx + 1 > n:
            break
        else:
            articles.append(utilities.fetch(domain+link))
    
    # excludes any video, audio results
    # search result links are relevant links, convert them to absolute links by adding the domain

    return articles

In [113]:
def get_articles_bag_of_words(articles):
    
    '''
        Return a list of words from a list of article htmls.
        
        get_articles_bag_of_words(articles) takes the html from the articles and return a list of words. In
        technical terms: for each article, find all the texts on the page (<p> using BeautifulSoup),
        extract words, clean words, return a list of words.
    
    '''
    
    bag_of_words = []
    for article in articles:
        # extract words from html and append to bag of words here
        soup = BeautifulSoup(article)
        for idx, para in enumerate(soup.find_all('p')):
            #             print(para.text)
            #             print(para.text.split(" "))
            for word in para.text.split(" "):
                if word[0:4] == 'http':
                    continue
                if word[-2:-1] == ':':
                    continue
                if word.isalpha():
                    bag_of_words.append(word)

    return bag_of_words

In [114]:
def get_sentiment(words):
    
    '''
    DO NOT MODIFY THIS FUNCTION
    
    Takes a list of words and returns a sentimental score. 
    
    '''
    
    analyzer = SentimentIntensityAnalyzer()
    total_compound = 0
    
    for w in words:
        score = analyzer.polarity_scores(w)
        total_compound += score['compound']
        
    return total_compound

In [115]:
def get_decision(tweets_sentiments, articles_sentiments):
    
    '''
    DO NOT MODIFY THIS FUNCTION
    
    Takes two sentimental scores from tweets and articles and return a final financial decision. 
    Invest if total sentiment is positive. 
    
    '''
    
    return (tweets_sentiments + articles_sentiments > 0)

### Results

In [116]:
company = Company('facebook')

In [117]:
company.tweets = get_tweets('facebook', 200)

{'created_at': 'Fri Nov 10 14:52:54 +0000 2017', 'id': 928998907862011904, 'id_str': '928998907862011904', 'text': 'RT @HeymanHustle: . @BrockLesnar vs @AJStylesOrg \n@WWE #SurvivorSeries \n#ChampionvsChampion \nMeanwhile, over on #Facebook - https://t.co/n2…', 'truncated': False, 'entities': {'hashtags': [{'text': 'SurvivorSeries', 'indices': [55, 70]}, {'text': 'ChampionvsChampion', 'indices': [72, 91]}, {'text': 'Facebook', 'indices': [112, 121]}], 'symbols': [], 'user_mentions': [{'screen_name': 'HeymanHustle', 'name': 'Paul Heyman', 'id': 23089228, 'id_str': '23089228', 'indices': [3, 16]}, {'screen_name': 'BrockLesnar', 'name': 'Brock Lesnar', 'id': 1222616833, 'id_str': '1222616833', 'indices': [20, 32]}, {'screen_name': 'AJStylesOrg', 'name': 'AJStyles.Org', 'id': 43769311, 'id_str': '43769311', 'indices': [36, 48]}, {'screen_name': 'WWE', 'name': 'WWE', 'id': 7517222, 'id_str': '7517222', 'indices': [50, 54]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type'

In [118]:
company.articles = get_article('facebook', 3)

Retrieving from cache: https://www.usatoday.com/search/facebook/
Retrieving from cache: http://www.usatoday.com/story/news/2013/01/09/corrections-clarifications/1821023/
Retrieving from cache: http://www.usatoday.com/story/news/2013/01/09/corrections-clarifications/1821023/
Retrieving from cache: http://www.usatoday.com/story/tech/2017/11/09/facebook-friends-small-businesses-facebook-community-boost/847094001/


/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [119]:
company.tweets_sentiments = get_sentiment(company.tweets)

In [120]:
company.articles_sentiments = get_sentiment(company.articles)

In [121]:
company.investment = get_decision(company.tweets_sentiments, company.articles_sentiments)

In [122]:
print(company)

<class '__main__.Company'>: {'name': 'facebook', 'articles': ['Settings', 'Cancel', 'Set', 'Hi', 'Already', 'a', 'Subscribe', 'to', 'USA', 'TODAY', 'Already', 'a', 'print', 'edition', 'but', 'have', 'a', 'Manage', 'your', 'account', 'Support', 'Log', 'Out', 'Get', 'the', 'news', 'Let', 'friends', 'in', 'your', 'social', 'network', 'know', 'what', 'you', 'are', 'reading', 'about', 'The', 'following', 'corrections', 'and', 'clarifications', 'have', 'been', 'published', 'by', 'USA', 'A', 'link', 'has', 'been', 'posted', 'to', 'your', 'Facebook', 'To', 'report', 'corrections', 'contact', 'USA', 'TODAY', 'Network', 'Standards', 'Editor', 'Brent', 'Please', 'indicate', 'whether', 'responding', 'to', 'content', 'online', 'or', 'in', 'the', 'The', 'following', 'corrections', 'clarifications', 'have', 'been', 'published', 'on', 'stories', 'produced', 'by', 'USA', 'November', 'item', 'in', 'the', 'Michigan', 'section', 'of', 'the', 'page', 'mistakenly', 'contained', 'information', 'about', 'a', 